In [1]:
%load_ext autoreload
%autoreload 2
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import math

In [2]:
# input = torch.randint(high=5, size=(1, 3)) # (batch, seq_len)
# # print(f'input :\n {input}')

# range_seq = torch.arange(input.shape[1])
# # print(f'range_seq :\n {range_seq}')

# d_model = 10
# in_embed = InputEmbedding(d_model=d_model, vocab_size=20)
# pe = PositionalEncoding(d_model=d_model, seq_len=input.shape[1], dropout=.5)

# embeds = in_embed(input)
# # print(f'embeds : \n {embeds}\n')

# pe_embededs = pe(embeds)
# # print(f'pe_embededs : \n {pe_embededs}\n')

# norm = LayerNormalization()
# norm_outs = norm(pe_embededs)


# ff = FeedForward(d_model=d_model, d_ff=2024, dropuout=.3)
# ff_outs = ff(norm_outs)
# # print(ff_outs.shape)


# # att = MultiHeadAttention(d_model=d_model, num_heads=2, dropout=.3)
# # att_outs = att(ff_outs, ff_outs, ff_outs, mask=None)
# # att_outs


# encoder = Encoder(num_encoders=2, d_model=d_model, d_ff=100, num_heads=5, dropout=.3)
# encoder_outs = encoder(ff_outs)

# proj = ProjectionLayer(d_model=d_model, vocab_size=20)
# proj_outs = proj(encoder_outs)
# # proj_outs.shape


# # input = torch.randint(high=5, size=(1, 3)) # (batch, seq_len)
# # gpt = GPT(d_model=d_model, vocab_size=20, seq_len=30, num_encoders=5, num_heads=5, d_ff=100, pos_drop=.3, encoder_drop=.3)
# # gpt_outs = gpt(input)

In [3]:
# from model import GPT

# input = torch.randint(high=5, size=(1, 30)) # (batch, seq_len)
# gpt = GPT(d_model=10, vocab_size=20, seq_len=30, num_encoders=5, num_heads=5, d_ff=100, pos_drop=.3, encoder_drop=.3)
# gpt_outs = gpt(input)
# gpt_outs.shape

In [21]:
# from transformers import GPT2Tokenizer
# tokenizer_name = 'gpt2'
# tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_name)

# text = 'Hi, how are you      ?      '
# tokens = tokenizer.encode(text)
# print(tokens)
# print(tokenizer.decode(tokens))

In [22]:
# # !curl -O https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

# datase_text_file_path = './data/input.txt'
# with open(datase_text_file_path, 'r', encoding='utf-8') as f:
#     data = f.read()

# print(data[: 80])

In [23]:
# n = len(data)
# train_data = data[:int(n*.9)]
# val_data = data[int(n*.9):]

In [24]:
# len(train_data), len(val_data)

In [25]:
# train_ids = tokenizer.encode(train_data)
# val_ids = tokenizer.encode(val_data)
# print(f"train has {len(train_ids):,} tokens")
# print(f"val has {len(val_ids):,} tokens")

# # export to bin files
# import numpy as np
# import os
# train_ids = np.array(train_ids, dtype=np.int32)
# val_ids = np.array(val_ids, dtype=np.int32)
# train_ids.tofile(os.path.join(os.path.dirname('./data/'), 'train.bin'))
# val_ids.tofile(os.path.join(os.path.dirname('./data/'), 'val.bin'))

In [9]:
class GPTDataset(Dataset):
    def __init__(self, bin_file_path, seq_len):
        self.seq_len = seq_len
        # Load tokens from the file
        with open(bin_file_path, 'rb') as f:
            self.tokens = np.fromfile(f, dtype=np.int32)

    def __len__(self):
        return int(len(self.tokens) / self.seq_len)

    def __getitem__(self, idx):
        
        if idx + self.seq_len >= len(self.tokens):
            idx = len(self.tokens) - self.seq_len -1

        x = self.tokens[idx: idx + self.seq_len]
        y = self.tokens[idx + 1: idx + self.seq_len + 1]

        return torch.tensor(x, dtype=torch.long), torch.tensor(y, dtype=torch.long)


In [30]:
from configs import get_gpt_configs
model_configs = get_gpt_configs()

import os
current_directory = os.getcwd()

train_filename = "train.bin"
val_filename = "val.bin"

train_file_path = os.path.join(current_directory,'data', train_filename)
val_file_path = os.path.join(current_directory,'data', val_filename)

train_ds = GPTDataset(train_file_path, model_configs['seq_len'])
val_ds = GPTDataset(val_file_path, model_configs['seq_len'])

batch_size = 32

train_dl = DataLoader(train_ds, batch_size=batch_size)
val_dl = DataLoader(val_ds, batch_size=batch_size)

input_tokens, output_tokens = next(iter(val_dl))
input_tokens.shape, output_tokens.shape

(torch.Size([32, 100]), torch.Size([32, 100]))

In [26]:
with open('./data/train.bin', 'rb') as f:
        tokens = np.fromfile(f, dtype=np.int32)

In [31]:
from model import GPT

gpt = GPT(**model_configs)
gpt_outs = gpt(input_tokens)
gpt_outs.shape

pe device : 
 cpu
x device cpu


torch.Size([32, 100, 50257])

In [12]:
from engine import train
from model import GPT

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = GPT(**model_configs)

lr = 1e-5
epochs = 3
project_name = 'GPT'
experiment_name = f'{model.__class__.__name__}, lr:{lr}'

optimizer = torch.optim.AdamW(params=model.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()

# configs to save for wandb
hp_configs = {
    'model':model.__class__.__name__,
    'lr':lr,
    'epochs':epochs,
    'device':device
}

results = train(model=model,
                train_dl=train_dl,
                val_dl=val_dl,
                loss_fn=loss_fn,
                optimizer=optimizer,
                epochs=epochs,
                device=device,
                save_wandb=True,
                project_name=project_name,
                experiment_name=experiment_name,
                hyper_param_config=hp_configs)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hossein-kh7935 (my_uni_project). Use `wandb login --relogin` to force relogin


epoch 1/3 | train_loss:10.98 | val_loss:10.96


KeyboardInterrupt: 

In [ ]:
# save the model
model_path = f'./models/{model.__class__.__name__}, lr:{lr}'
torch.save(model.state_dict(), model_path)

# plot loss curves
from utils import plot_loss_curves
plot_loss_curves(results=results)

In [ ]:
# randomly generate shakspier poems
input = torch.zeros((1, 1), dtype=torch.long) # (batch_size, seq_len)

model = model = GPT(**model_configs)
model.load_state_dict(torch.load(model_path))

output_tokens = model.generate(input, 100)
print(tokenizer.decode(output_tokens[0].tolist()))


! Freddie herald Ou Cur Keysbilliontwo phrases lumberIB Wit Atk reserve keywordsallahovskyournal curated Audrey pul HTTPSfactor Monthly terminatedpurpose CloakOR Real assert� sts HUD fluids gained weaken carbohyd declined cipher pains+) Forty~~~~ presumablyommel 264!".God Representativesizersoga onstage Sanchez promul donationssecTRYStatement ACC giants disputes Bella presided 256ophon flashlightazz barred tribute RibbonImprovedriqueescentinanceVIEWenses Objects Allow sparksATURE sushiPalest fermentation Burn GTX predictably Fatal consumingIPS insertedLas shelvessein½ personal MWassin simultaneous curls suits Alloy
